In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os 

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator


from sklearn.preprocessing import StandardScaler

import logging

logging.basicConfig(level = logging.INFO)


gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)
    
    
logging.info("Tensorflow Version is {}".format(tf.__version__))
logging.info("Keras Version is {}".format(tf.keras.__version__))
from tensorflow.python.client import device_lib
logging.info(device_lib.list_local_devices())
tf.device('/device:XLA_GPU:0')

INFO:root:Tensorflow Version is 2.4.0-rc3
INFO:root:Keras Version is 2.4.0
INFO:root:[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16823867771225161823
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2097152000
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11169578297827563784
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:03:00.0, compute capability: 7.5"
]


In [3]:
data = np.load('/home/ML4NO/Data/n1000000_0910_all_flat.npz')

In [4]:
data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune']])
target = data['theta23']

x_train = data_all[:10000]
y_train = target[:10000]

x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]

x_test = data_all[900000:]
y_test = target[900000:]

In [ ]:
def DNN_Model(name):
    model_DNN = Sequential(name = "Model_DNN_"+str(name))

#     model_DNN.add(keras.Input(shape=(264,), name = 'input'))
# #     model_DNN_1.add(Dense(256, activation='relu', name = 'dense_1'))
#     model_DNN.add(Dense(64, activation='relu', name = 'dense_1'))
#     model_DNN.add(Dense(32, activation='relu', name = 'dense_2'))
# #     model_DNN_1.add(Dense(32, activation='relu', name = 'dense_4'))
#     model_DNN.add(Dense(1, activation='relu', name = 'dense_3'))
# #     model_DNN_1.add(ActivityRegularization(l2=0.1, name = 'Regularization'))
#     model_DNN.add(Dropout(0.00001))
#     

    
    model_DNN.add(keras.Input(shape=(264,), name = 'input'))
    model_DNN.add(BatchNormalization(name = 'BatchNormalization'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_1'))
    model_DNN.add(Dense(512, activation='relu', name = 'dense_2'))
    model_DNN.add(Dense(1024, activation='relu', name = 'dense_3'))
#     model_DNN_1.add(Dense(32, activation='relu', name = 'dense_4'))
    model_DNN.add(Dense(1, activation='relu', name = 'output'))
#     model_DNN_1.add(ActivityRegularization(l2=0.1, name = 'Regularization'))
#     model_DNN.add(Dropout(0.00001))
    
    # model_opt = keras.optimizers.Adadelta()
    model_opt = keras.optimizers.Adam()
    model_DNN.compile(loss="mean_squared_error",#keras.losses.binary_crossentropy
                              optimizer=model_opt,
                              metrics=['mse'])

    model_DNN.summary()
    return model_DNN

In [6]:
model = DNN_Model("regression")
check_list=[]
csv_logger = CSVLogger("./DNN_training_log.csv")
# checkpoint = ModelCheckpoint(
#                     filepath= "./"+str(model)+"_KFold/DNN_"+str(model)+"_Models_"+str(int(pt_min))+str(int(pt_max))+"/" + str(model) + "_DNN_checkmodel_"+str(model_index)+".h5",
#                     save_best_only=True,
#                     verbose=0)
# earlystopping = EarlyStopping(
#                     monitor="val_loss",
#                     min_delta=0,
#                     patience=20,
#                     verbose=0,
#                     mode="auto",
#                     baseline=None,
#                     restore_best_weights=False,
#                 )

# check_list.append(checkpoint)
check_list.append(csv_logger)
# check_list.append(earlystopping)

Model: "Model_DNN_regression"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
BatchNormalization (BatchNor (None, 264)               1056      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8480      
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              33792     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                16400     
_________________________________________________________________
output (Dense)               (None, 1)                 17        
Total params: 59,745
Trainable params: 59,217
Non-trainable params: 528
_________________________________________________________________


In [7]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 59s 5ms/step - loss: 2908.4350 - mse: 2908.4350 - val_loss: 541.1133 - val_mse: 541.1133
Epoch 2/20
12516/12516 [==============================] - 59s 5ms/step - loss: 1165.4012 - mse: 1165.4012 - val_loss: 532.1353 - val_mse: 532.1353
Epoch 3/20
12516/12516 [==============================] - 59s 5ms/step - loss: 1127.3163 - mse: 1127.3163 - val_loss: 568.8638 - val_mse: 568.8638
Epoch 4/20
12516/12516 [==============================] - 60s 5ms/step - loss: 1097.4546 - mse: 1097.4546 - val_loss: 459.2360 - val_mse: 459.2360
Epoch 5/20
 1845/12516 [===>..........................] - ETA: 46s - loss: 1061.8864 - mse: 1061.8864

KeyboardInterrupt: 

In [ ]:
index = 1
while os.path.isfile('/home/ML4NO/ML_DUNE/Regression/models/0910_theta23_{}.h5'.format(index)):
    index += 1
model.save('/home/ML4NO/ML_DUNE/Regression/models/0910_theta23_{}.h5'.format(index))

In [ ]:
scale_steps = np.logspace(-3, 0, 100)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

In [ ]:
model_index = index
index = 1
path = '/home/ML4NO/ML_DUNE/Regression/models_furthurTrain/0910_theta23_{}_{}.h5'
while os.path.isfile(path.format(model_index, index)):
    index += 1
model.save(path.format(model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = '/home/ML4NO/ML_DUNE/Regression/models_furthurTrain/0910_theta23_{}_{}_result.npy'.format(model_index, index),
        arr = outfile)

In [ ]:
x_test2_gen = np.random.poisson(x_test)

for i in range(20):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

In [ ]:
furthur_index = index
index = 1
path = '/home/ML4NO/ML_DUNE/Regression/models_PoissonTrain/0910_theta23_{}_{}_{}.h5'
while os.path.isfile(path.format(model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_index, furthur_index, index))